# 敵対的生成ネットワーク(Generative Adversarial Network : GAN)

敵対的生成ネットワーク(GAN)は確率的モデリングの強力なアプローチです(I. Goodfellow et al., 2014; I. Goodfellow, 2016)。
このモデルでは深層生成モデルを家庭し、速く正確な推論が可能になります。

ここではEdwardでの例を示します。原文のWebページ版は　http://edwardlib.org/tutorials/gan　です。

In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import edward as ed
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import os
import tensorflow as tf

from edward.models import Uniform
from observations import mnist
from tensorflow.contrib import slim

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def plot(samples):
  fig = plt.figure(figsize=(4, 4))
  gs = gridspec.GridSpec(4, 4)
  gs.update(wspace=0.05, hspace=0.05)

  for i, sample in enumerate(samples):
    ax = plt.subplot(gs[i])
    plt.axis('off')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_aspect('equal')
    plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

  return fig

In [ ]:
ed.set_seed(42)

data_dir = "/tmp/data"
out_dir = "/tmp/out"

M = 128  # バッチサイズ
d = 100  # 隠れ変数の次元

# Data

55,000 枚の$28\times　28$画像で構成されたMNIST(LeCun, Bottou, Bengio, & Haffner, 1998)のデータを使います。
それぞれの画像はflatにされた784次元ベクトルで表現されそれぞれの要素は0から1の間のピクセル値です。

![GAN Fig 0](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/gan-fig0.png)

このノートでのゴールは高い品質の手書き数字画像を生成するモデルを推定することです。

訓練の間にMNISTの数字のバッチを取り込むことになります。tensorflowのplaceholderを固定したバッチサイズ$M$枚の画像に対してインスタンス化します。

また例となるデータ全体の集合からから次のバッチのデータ点を選び出すヘルパー関数を定義します。
これはある時点で処理中のバッチのインデックス(番号）を保持しておいて次のバッチを関数``next()``を使って返すものです。
推測処理の間`x_train_generator`を使ってバッチを生成することになります。

In [ ]:
def generator(array, batch_size):
  """Generate batch with respect to array's first axis."""
  start = 0  # pointer to where we are in iteration
  while True:
    stop = start + batch_size
    diff = stop - array.shape[0]
    if diff <= 0:
      batch = array[start:stop]
      start += batch_size
    else:
      batch = np.concatenate((array[start:], array[:diff]))
      start = diff
    batch = batch.astype(np.float32) / 255.0  # normalize pixel intensities
    batch = np.random.binomial(1, batch)  # binarize images
    yield batch

In [ ]:
(x_train, _), (x_test, _) = mnist(data_dir)
x_train_generator = generator(x_train, M)
x_ph = tf.placeholder(tf.float32, [M, 784])

# Model

GANは暗黙的な方法で生成モデルを仮定しています。与えられたランダムノイズに対しデータはその決定論的な関数として生成されるものとされます。

形式的には生成の過程は$G(\cdot; \theta)$をサンプル$\mathbf{\epsilon}$を入力としてとるニューラルネットとして

\begin{align*}
\mathbf{\epsilon} &\sim p(\mathbf{\epsilon}), \\
\mathbf{x} &= G(\mathbf{\epsilon}; \theta),
\end{align*}

と書けます。分布$p(\mathbf{\epsilon})$は物理的システムに対して確率的な性質を入れるランダムノイズとして解釈されます。
これは典型的には隠れた変数と同じ次元の一様分布または正規分布になります。

EdwardではTensorFlow Slimを使ってニューラルネットを指定することで以下のようなモデルを作ります。
これは２層全結合のニューラルネットと$[0,1]$間に値をとる$28\times28$次元のベクトル出力として定義されます。


In [ ]:
def generative_network(eps):
  h1 = slim.fully_connected(eps, 128, activation_fn=tf.nn.relu)
  x = slim.fully_connected(h1, 784, activation_fn=tf.sigmoid)
  return x

with tf.variable_scope("Gen"):
  eps = Uniform(tf.zeros([M, d]) - 1.0, tf.ones([M, d]))
  x = generative_network(eps)

生成的ネットワークを用いてデータの特徴を最もよく捉えるモデルのパラメーターを推定しようと思います。
(GANを使う場合はパラメーターの推定が興味の対象で、隠れた変数の推定ではないことに注意)

不幸なことに上で書いた確率モデルは取り扱いやすい尤度が認められません。
これではモデルの密度を必要とするような多くの推測アルゴリズムは使えません。
そこでここではモデルから得られるサンプルのみを仮定する尤度自由("likelihood-free")アルゴリズム(Marin, Pudlo, Robert, & Ryder, 2012)
を用いようと思います。


# 推測

尤度自由アルゴリズムの発想の本質はモデルからのサンプルと真のデータの分布からのサンプルの間の不一致性(discrepancy)を解析することによる
比較による学習です (e.g., Rubin (1984; Gretton, Borgwardt, Rasch, Schölkopf, & Smola, 2012))。
モデルがより良いサンプルを生成するためにどこを改良すれば良いかの情報を持っています。
    
GANにおいてはニューラルネット$D(\cdot;\phi)$がこの比較を行い、識別器(discriminator)として知られています。
$D(\cdot;\phi)$はデータ$\mathbf{x}$を(モデルまたはデータの集合から取られたデータ点からの生成の)入力として取り、
が真のデータから来る確率$\mathbf{x}$を計算します。

Edwardでは以下のような識別を使います。これはと1つのReLU隠れ層をもつフィードフォワードネットワークです。
logit関数を適用した(拘束のない)スケールでの確率を返します。

In [ ]:
def discriminative_network(x):
  """Outputs probability in logits."""
  h1 = slim.fully_connected(x, 128, activation_fn=tf.nn.relu)
  logit = slim.fully_connected(h1, 1, activation_fn=None)
  return logit

$p^*(\mathbf{x})$を真のデータ分布を表現するものとするとGANを用いた最適化問題は

\begin{equation*}
\min_\theta \max_\phi~
\mathbb{E}_{p^*(\mathbf{x})} [ \log D(\mathbf{x}; \phi) ]
+ \mathbb{E}_{p(\mathbf{x}; \theta)} [ \log (1 - D(\mathbf{x}; \phi)) ].
\end{equation*}

と書けます。この最適化問題は2つの手順からなっています。
生成されたパラメータに従って最小の解を求める手順と識別パラメータに基づいて最大の解を求める手順です。

実際にはアルゴリズムは勾配の更新を繰り返すことで進みます。
追加の経験的な方法として生成モデルの目的関数を勾配の飽和を防ぐために変更するというのがあります(I. J. Goodfellow, 2014)。

多くの直感的解釈の源がGAN的な訓練には存在します。
その一つはこの方法の元々の動機の一つですが2つのニューラルネットがゲームをしているというアイデアに基づくものがあります。
識別きは生成器から出されるサンプルを最もよく識別するようにし、生成器はなるべく識別器に識別されないようなサンプルを生成するようにするというものです。
訓練のゴールはナッシュ均衡を実現することです。

別のアイデアの源泉は教師あり学習としての教師なし学習です(M. U. Gutmann, Dutta, Kaski, & Corander, 2014; M. Gutmann & Hyvärinen, 2010)。
これは識別問題—近年の問題は比較的に簡単な問題とされる—の力にレバレッジをかけるものです　。

3つ目は古典的統計学の識別器は密度比の代理と解釈できるという点からきます
 (Mohamed & Lakshminarayanan, 2016; Sugiyama, Suzuki, & Kanamori, 2012)。
元々の問題(最大尤度のような)識別器とモデルの密度を必要とするものを水増しすることで識別器が最適なものとなった場合は元の問題を再現できることになります。
さらにこの近似はとても速く、近似推論としての見方からGANに正当性を与えています。

EdwardではGANのアルゴリズム(`GANInference`)は単純に`x`とその実現`x_ph`であるを入力とした暗黙的な密度モデルを取ります。
それに加えパラメトライズされた関数`discriminator`はサンプルを識別するために与えられます。


inference = ed.GANInference(
    data={x: x_ph}, discriminator=discriminative_network)

ここではADAMを生成器と識別器の両方の最適化に用います。
15,000回のイテレーションを行い、1000回おきに進捗を表示します。


In [ ]:
optimizer = tf.train.AdamOptimizer()
optimizer_d = tf.train.AdamOptimizer()

inference = ed.GANInference(
    data={x: x_ph}, discriminator=discriminative_network)
inference.initialize(
    optimizer=optimizer, optimizer_d=optimizer_d,
    n_iter=15000, n_print=1000)

以下でGANを学習させるメインループを書きます。
各イテレーションにおいてミニバッチが取り入れられパラメータがアルゴリズム(の結果）に基づいて更新されます。
1000回のイテレーションごとに進捗を表示するとともにモデルから生成されたサンプルの画像を保存します。

In [ ]:
sess = ed.get_session()
tf.global_variables_initializer().run()

idx = np.random.randint(M, size=16)
i = 0
for t in range(inference.n_iter):
  if t % inference.n_print == 0:
    samples = sess.run(x)
    samples = samples[idx, ]

    fig = plot(samples)
    plt.savefig(os.path.join(out_dir, '{}.png').format(
        str(i).zfill(3)), bbox_inches='tight')
    plt.close(fig)
    i += 1

  x_batch = next(x_train_generator)
  info_dict = inference.update(feed_dict={x_ph: x_batch})
  inference.print_progress(info_dict)

GANの目的関数の収束を試験することは実際上は意味を持たないでしょう。
このアルゴリズムは大抵サンプルの見た目が大丈夫であるかあるいはGANがデータの意味のある部分を再現することができた
かなどある別の基準が満たされるまで走ることになります。

## Criticism

GANの評価は未解決の問題(open problem)として残されています。
データへの適合性の評価の観点からも、収束を達成しているかの点からも。

最近の進展では別の目的関数とヒューリスティックが学習を安定化させるために考案されています。
(Soumith Chintalaの　[GAN hacks repo](https://github.com/soumith/ganhacks)も参照).

一つのモデル評価のアプローチは学習の間に生成された画像を単に目で見流というものがあります。
以下に14,000回のイテレーション後に生成された画像を示します(これは14,000回生成器と識別器の勾配が更新されたものです)。

![GAN Fig 1](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/gan-fig1.png)
                                  
この画像は少しぼやけていますが、有意義なものです。
さらなる進展への提案として識別ネットワーク、生成ネットワークの容量を改善させるための
最適化におけるハイパーパラメーターの調整、(畳み込み構造などの)さらなる事前情報を取り入れることなどがあります。
